In [5]:
pip install backtrader

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 419.5/419.5 kB 3.0 MB/s eta 0:00:00a 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [10]:
import backtrader as bt 
import yfinance as yf 
import pandas as pd 
import numpy as np 

In [11]:
stocks = ['NVDA', 'GOOGL', 'GS' , 'GME', 'MSFT']
selected = 'GME'

In [12]:
import yfinance as yf
import pandas as pd

stock_earnings_data = {}
for stock in stocks:
    obj = yf.Ticker(stock)
    earnings_data = obj.get_earnings_dates(64)
    
    # Check if earnings_data is not None before processing
    if earnings_data is not None:
        # Convert index from America/New_York (UTC-5) to UTC
        earnings_data.index = earnings_data.index.tz_convert('UTC')
        
        # Keep only past earnings dates
        today = pd.Timestamp.now(tz='UTC').normalize()
        earnings_data = earnings_data[earnings_data.index <= today]
        
        stock_earnings_data[stock] = earnings_data
    else:
        print(f"No earnings data available for {stock}")

# Check if selected stock has data before accessing
if selected in stock_earnings_data:
    print(stock_earnings_data[selected])
else:
    print(f"No earnings data available for {selected}")

                           EPS Estimate  Reported EPS  Surprise(%)
Earnings Date                                                     
2025-03-25 00:00:00+00:00           NaN           NaN          NaN
2024-12-10 00:00:00+00:00         -0.06          0.06       194.49
2024-09-10 00:00:00+00:00         -0.08          0.01       111.76
2024-06-07 00:00:00+00:00         -0.09         -0.12       -33.33
2024-03-26 00:00:00+00:00          0.30          0.22       -25.42
2023-12-06 00:00:00+00:00         -0.09           NaN       100.00
2023-09-06 00:00:00+00:00         -0.14         -0.03        78.77
2023-06-07 00:00:00+00:00         -0.12         -0.14       -16.67
2023-03-21 00:00:00+00:00         -0.13          0.16       220.30
2022-12-07 00:00:00+00:00         -0.28         -0.31       -12.05
2022-09-07 00:00:00+00:00         -0.38         -0.35         7.28
2022-06-01 00:00:00+00:00         -0.36         -0.52       -43.45
2022-03-17 00:00:00+00:00          0.21         -0.46      -32

In [13]:
earliest_entries = {} 
stocks_price_data = {stock : None for stock in stocks}
import warnings
warnings.filterwarnings("ignore")
for stock in stocks : 
      stock_df = pd.read_csv(f'{stock}_Earnings_Data(5M).csv')
      stock_df['date'] = pd.to_datetime(stock_df['date'],format='%Y%m%d %H:%M:%S %Z', errors='coerce')
      stock_df = stock_df.loc[:, ~stock_df.columns.str.contains('^Unnamed')]
      stock_df.dropna(subset=['date'], inplace=True)
      earliest_date = stock_df['date'].min()
      stock_df.set_index('date', inplace=True)
      stock_df = stock_df.sort_index()
      stock_df = stock_df.drop(columns=['ReqId','ticker'])
      earliest_entries[stock] = earliest_date
      stocks_price_data[stock] = stock_df
print(stocks_price_data['GME'])

                            Open   High    Low  Close  Volume
date                                                         
2010-05-20 16:00:00-04:00   5.17   5.17   5.16   5.16  831200
2010-05-20 16:05:00-04:00   5.16   5.16   5.16   5.16    2000
2010-05-20 16:10:00-04:00   5.14   5.14   5.14   5.14    4000
2010-05-20 16:15:00-04:00   5.14   5.14   5.14   5.14    1200
2010-05-20 16:20:00-04:00   5.14   5.14   5.14   5.14       0
...                          ...    ...    ...    ...     ...
2024-12-10 18:05:00-05:00  27.91  28.09  27.71  27.85   31986
2024-12-10 18:10:00-05:00  27.89  28.09  27.61  27.74   81445
2024-12-10 18:15:00-05:00  27.74  27.85  27.60  27.62   27836
2024-12-10 18:20:00-05:00  27.63  27.82  27.62  27.78   15939
2024-12-10 18:25:00-05:00  27.70  27.80  27.69  27.80   11644

[2478 rows x 5 columns]


In [14]:
# Convert index to Date only (removing time)
stocks_price_data['GME']['DateOnly'] = stocks_price_data['GME'].index.date
stock_earnings_data['GME']['DateOnly'] = stock_earnings_data['GME'].index.date

# Merge on the new DateOnly column
merged_df = stocks_price_data['GME'].merge(stock_earnings_data['GME']['DateOnly'], on='DateOnly', how='inner')
merged_df.set_index('DateOnly',inplace=True)
merged_df = merged_df.sort_index()
print(len(merged_df.index.unique()))

# Drop the helper column if needed
# merged_df.drop(columns=['DateOnly'], inplace=True)





59


In [15]:
year_months_with_prices = {}

for stock, df in stocks_price_data.items():
    # Extract year and month where both 16:05 and 17:05 exist
    df['YearMonth'] = df.index.to_period('M')  # Convert to Year-Month format
    grouped = df.groupby('YearMonth').filter(lambda x: 
        '16:05' in x.index.strftime('%H:%M').values and '17:05' in x.index.strftime('%H:%M').values)

    # Store unique Year-Months where both timestamps exist
    year_months_with_prices[stock] = grouped['YearMonth'].unique().tolist()

# Display the available Year-Months for each stock
for val in year_months_with_prices.values() : 
    print(len(val))


59
38
60
59
60


In [ ]:
#Looking at the lenghts of the dataframes after filtering for our required timestamps and comparing them to the initial lengths and expected lengths
#We come to the conclusion that the data is inadequate, so we go ahead and revise our data collection code 
#

In [ ]:
# we will construct the backtest one by one for each stock since we have different function/regresssion definitions for each stock 

In [16]:
#how do we start? we start by taking data from 16:05 to 18:05 wherever possible 
filtered_data = {stock : None for stock in stocks} 
for stock,df in stocks_price_data.items() : 
      df['hour'] = df.index.hour
      df['minute'] = df.index.minute
      filtered_df = df[(df['hour'] >= 16) & (df['hour'] <= 18) & ((df['hour'] != 16) | (df['minute'] >= 5))]
      filtered_df.drop(columns=['hour', 'minute'], inplace=True)
      filtered_data[stock] = filtered_df

print(filtered_data[selected])
 


                            Open   High    Low  Close  Volume    DateOnly  \
date                                                                        
2010-05-20 16:05:00-04:00   5.16   5.16   5.16   5.16    2000  2010-05-20   
2010-05-20 16:10:00-04:00   5.14   5.14   5.14   5.14    4000  2010-05-20   
2010-05-20 16:15:00-04:00   5.14   5.14   5.14   5.14    1200  2010-05-20   
2010-05-20 16:20:00-04:00   5.14   5.14   5.14   5.14       0  2010-05-20   
2010-05-20 16:25:00-04:00   5.14   5.14   5.14   5.14       0  2010-05-20   
...                          ...    ...    ...    ...     ...         ...   
2024-12-10 18:05:00-05:00  27.91  28.09  27.71  27.85   31986  2024-12-10   
2024-12-10 18:10:00-05:00  27.89  28.09  27.61  27.74   81445  2024-12-10   
2024-12-10 18:15:00-05:00  27.74  27.85  27.60  27.62   27836  2024-12-10   
2024-12-10 18:20:00-05:00  27.63  27.82  27.62  27.78   15939  2024-12-10   
2024-12-10 18:25:00-05:00  27.70  27.80  27.69  27.80   11644  2024-12-10   

In [17]:
price_dates = filtered_data[selected].index.normalize().unique()
# Extract unique earnings release dates
earnings_dates = stock_earnings_data[selected].index.normalize().unique()

# Find missing earnings dates (dates in earnings but not in price data)
missing_earnings_dates = set(earnings_dates) - set(price_dates)
print(missing_earnings_dates)


{Timestamp('2015-05-28 00:00:00+0000', tz='UTC'), Timestamp('2011-03-24 00:00:00+0000', tz='UTC'), Timestamp('2012-03-22 00:00:00+0000', tz='UTC'), Timestamp('2023-03-21 00:00:00+0000', tz='UTC'), Timestamp('2015-03-26 00:00:00+0000', tz='UTC'), Timestamp('2018-05-31 00:00:00+0000', tz='UTC'), Timestamp('2024-12-10 00:00:00+0000', tz='UTC'), Timestamp('2023-12-06 00:00:00+0000', tz='UTC'), Timestamp('2019-04-02 00:00:00+0000', tz='UTC'), Timestamp('2024-09-10 00:00:00+0000', tz='UTC'), Timestamp('2016-05-26 00:00:00+0000', tz='UTC'), Timestamp('2023-06-07 00:00:00+0000', tz='UTC'), Timestamp('2021-09-08 00:00:00+0000', tz='UTC'), Timestamp('2010-11-18 00:00:00+0000', tz='UTC'), Timestamp('2013-08-22 00:00:00+0000', tz='UTC'), Timestamp('2013-11-21 00:00:00+0000', tz='UTC'), Timestamp('2017-11-21 00:00:00+0000', tz='UTC'), Timestamp('2010-05-20 00:00:00+0000', tz='UTC'), Timestamp('2016-08-25 00:00:00+0000', tz='UTC'), Timestamp('2013-05-23 00:00:00+0000', tz='UTC'), Timestamp('2019-09-

In [19]:
# import seaborn as sns
# import numpy as np

# # Convert index to datetime for grouping
# filtered_data[selected]['hour'] = filtered_data['MSFT'].index.hour
# filtered_data[selected]['day'] = filtered_data['MSFT'].index.date

# # Pivot Data for Heatmap
# heatmap_data = filtered_data[selected].pivot_table(index='hour', columns='day', values='Close', aggfunc=np.count_nonzero)

# # Plot Heatmap
# plt.figure(figsize=(14, 6))
# sns.heatmap(heatmap_data, cmap="coolwarm", cbar=True, linewidths=0.5)

# plt.title(f"{selected} Data Availability Heatmap", fontsize=14)
# plt.xlabel("Day", fontsize=12)
# plt.ylabel("Hour", fontsize=12)

# plt.show()


In [26]:
%matplotlib inline
import backtrader as bt
import backtrader.analyzers as btanalyzers
import datetime
import pandas as pd
import matplotlib.pyplot as plt



price_df = filtered_data[selected] 
earnings_df = stock_earnings_data[selected]

# Ensure price_df and earnings_df have no timezone
price_df.index = price_df.index.tz_localize(None)
earnings_df.index = earnings_df.index.tz_localize(None)


class EarningsTradingStrategy(bt.Strategy):
    params = (
        ('take_profit', 0.015),   # 1.5% Take Profit
        ('stop_loss', 0.015),      # 10% Stop Loss (adjust as needed)
        ('holding_period', 24),   # 2 hours in 5-min bars
        ('max_trade_value', 1000), # $1,000 max per trade
        ('regression_file', 'regression_predictions.csv'),
        ('date_threshold_days', 35),
        ('stock', None)
        
    )

    def __init__(self):
        self.order = None
        self.entry_price = None
        self.bar_count = 0
        self.trade_outcomes = []
        self._closed_by_tp = False
        self._closed_by_sl = False
        self.is_long = None  # Track position type: True=Long, False=Short
        self.portfolio_value = [] 

        # Use the symbol passed in parameters rather than extracting from data feed.
        self.symbol = self.p.stock

        # Load regression predictions from CSV, parsing 'Earnings_Date' as datetime.
        self.reg_preds = pd.read_csv(self.p.regression_file, parse_dates=['Earnings_Date'])
        # Filter predictions for this symbol.
        self.reg_preds = self.reg_preds[self.reg_preds['Symbol'] == self.symbol].copy()
        # Set Earnings_Date as index.
        self.reg_preds.set_index('Earnings_Date', inplace=True)
        # Pre-calculate the date threshold.
        self.date_threshold = pd.Timedelta(days=self.p.date_threshold_days)

    def notify_trade(self, trade):
        if trade.isclosed:
            outcome = 'timeout'
            if self._closed_by_tp:
                outcome = 'tp'
            elif self._closed_by_sl:
                outcome = 'sl'
            self.trade_outcomes.append((outcome, 'long' if self.is_long else 'short'))
            print(f"Trade closed by {outcome} ({'Long' if self.is_long else 'Short'})")

    def next(self):
        current_time = self.datas[0].datetime.time()
        current_date = pd.Timestamp(self.data.datetime.date(0)).normalize()
        
        # --- Fuzzy matching for regression prediction earnings date ---
        matched_row = None
        norm_preds = self.reg_preds.index.normalize()
        if current_date in norm_preds:
            matched_row = self.reg_preds.loc[current_date]
            if isinstance(matched_row, pd.DataFrame):
                matched_row = matched_row.iloc[0]
        else:
            if not norm_preds.empty:
                deltas = np.abs(norm_preds - current_date)
                min_delta = deltas.min()
                if min_delta <= self.date_threshold:
                    matched_row = self.reg_preds.iloc[deltas.argmin()]
        
        if matched_row is not None:
            predicted_eps = matched_row['Predicted_EPS']
        else:
            predicted_eps = None

        if datetime.time(16, 0) <= current_time <= datetime.time(16, 10) and not self.order:
            earnings = earnings_df.loc[earnings_df.index.date == current_date]
            if not earnings.empty:  
                # estimated_eps = earnings['EPS Estimate'].values[0]
                # reported_eps = earnings['Reported EPS'].values[0]
                # surprise = (reported_eps - estimated_eps) / estimated_eps
                estimated_eps = earnings['EPS Estimate'].values[0]
                reported_eps = earnings['Reported EPS'].values[0]
                estimate_surprise = (reported_eps - estimated_eps) / estimated_eps if estimated_eps != 0 else 0
                reg_surprise = (reported_eps - predicted_eps) / predicted_eps if predicted_eps != 0 else 0
                surprise = 0.2 * reg_surprise + 0.8 * estimate_surprise


                # Calculate position size based on $1,000 trade limit
                current_price = self.data.close[0]
                position_size = int(self.params.max_trade_value / current_price)  # Round to whole shares

                print(f"Date: {current_date}, Price: {current_price:.2f}, Position Size: {position_size}")

                if position_size <= 0:
                    print(f"Skipping trade on {current_date}: Position size is zero or negative.")
                    return

                if surprise >= 0.1:  # Long entry
                    self.order = self.buy(size=position_size)
                    self.is_long = True
                    print(f"LONG ENTRY at {current_price:.2f} on {current_date}")
                elif surprise <= -0.1:  # Short entry
                    self.order = self.sell(size=position_size)
                    self.is_long = False
                    print(f"SHORT ENTRY at {current_price:.2f} on {current_date}")
                
                if self.order:
                    self.entry_price = current_price
                    self.bar_count = 0

        if self.order and self.order.status == bt.Order.Completed:
            self.bar_count += 1
            price = self.data.close[0]
            
            # Calculate price change based on position type
            if self.is_long:
                price_change = (price - self.entry_price) / self.entry_price
                tp_condition = price_change >= self.params.take_profit
                sl_condition = price_change <= -self.params.stop_loss
            else:  # Short position
                price_change = (self.entry_price - price) / self.entry_price
                tp_condition = price_change >= self.params.take_profit
                sl_condition = price_change <= -self.params.stop_loss

            if tp_condition:
                print(f"Closing {'Long' if self.is_long else 'Short'} via Take Profit")
                self._closed_by_tp = True
                self.close()
                self.order = None
            elif sl_condition:
                print(f"Closing {'Long' if self.is_long else 'Short'} via Stop Loss")
                self._closed_by_sl = True
                self.close()
                self.order = None
            elif self.bar_count >= self.params.holding_period:
                print(f"Closing {'Long' if self.is_long else 'Short'} via Timeout")
                self.close()
                self.order = None
        self.portfolio_value.append(self.broker.getvalue())


                
cerebro = bt.Cerebro()
data = bt.feeds.PandasData(dataname=price_df, datetime=None)
cerebro.adddata(data)
cerebro.addstrategy(EarningsTradingStrategy, stock=selected, regression_file='../regression_predictions.csv', date_threshold_days=35)

cerebro.broker.setcash(10000.0)
cerebro.broker.setcommission(commission=0.001)
cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return', timeframe=bt.TimeFrame.Days)
cerebro.addanalyzer(btanalyzers.SharpeRatio, _name='mysharpe', riskfreerate=0.03, timeframe=bt.TimeFrame.Days, annualize=True)
# cerebro.addanalyzer(btanalyzers.SortinoRatio, _name='sortino', riskfreerate=0.0, timeframe=bt.TimeFrame.Days, annualize=True)
cerebro.addanalyzer(btanalyzers.AnnualReturn, _name='annual_return')
cerebro.addanalyzer(btanalyzers.DrawDown, _name='drawdown')
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='trade_analyzer')
cerebro.addanalyzer(btanalyzers.Returns, _name='returns')
cerebro.addanalyzer(btanalyzers.PyFolio, _name='pyfolio')  # For advanced metrics

thestrats = cerebro.run()
thestrat = thestrats[0]
sharpe_ratio = thestrat.analyzers.mysharpe.get_analysis()
trade_analysis = thestrat.analyzers.trade_analyzer.get_analysis()
time_return = thestrat.analyzers.time_return.get_analysis()
print("Sharpe Ratio:", sharpe_ratio['sharperatio'])
print(trade_analysis)

Sharpe Ratio: None
AutoOrderedDict({'total': AutoOrderedDict({'total': 0})})


In [27]:
print(f"Sharpe Ratio for {selected}", sharpe_ratio['sharperatio'])

Sharpe Ratio for GME None


In [ ]:
pip install -U kaleido


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\athabit\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
# import pandas as pd
# import plotly.express as px
# import os
# import datetime

# # Create directory for saving results
# def create_results_dir(base_path=None):
#     """
#     Creates a directory for saving results based on current timestamp
#     Returns the path to the created directory
#     """
#     # If no base path provided, use the directory where the notebook is located
#     if base_path is None:
#         base_path = os.path.dirname(os.path.abspath('Backtest.ipynb'))
    
#     # Create a timestamp-based folder name
#     timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
#     results_dir = os.path.join(base_path, f"GME_backtest_results_{timestamp}")
    
#     # Create the directory if it doesn't exist
#     if not os.path.exists(results_dir):
#         os.makedirs(results_dir)
        
#     return results_dir

# # Function to flatten AutoOrderedDict
# def flatten_dict(d, parent_key='', sep='.'):
#     """
#     Recursively flattens a nested dictionary.
#     """
#     items = []
#     for k, v in d.items():
#         new_key = f"{parent_key}{sep}{k}" if parent_key else k
#         if isinstance(v, dict):
#             items.extend(flatten_dict(v, new_key, sep=sep).items())
#         else:
#             items.append((new_key, v))
#     return dict(items)

# # Function to safely extract a metric value
# def get_metric_value(trade_df, metric_key):
#     """
#     Safely extracts a metric value from the DataFrame.
#     Returns None if the metric is not found.
#     """
#     matches = trade_df[trade_df['Metric'].str.contains(metric_key)]
#     if not matches.empty:
#         return matches['Value'].values[0]
#     return None

# # Create results directory
# results_dir = create_results_dir()
# print(f"Results will be saved to: {results_dir}")

# # Flatten the trade_analysis object
# flat_trade_analysis = flatten_dict(trade_analysis)

# # Convert the flattened dictionary to a DataFrame
# trade_df = pd.DataFrame(list(flat_trade_analysis.items()), columns=['Metric', 'Value'])

# # Save the full analysis to CSV
# trade_df.to_csv(os.path.join(results_dir, "trade_analysis.csv"), index=False)
# print(f"Trade analysis saved to CSV")

# # Print the DataFrame
# print(trade_df)

# # 1. Total Trades
# total_trades = trade_df[trade_df['Metric'].str.contains('total.total')]['Value'].values[0]
# won_trades = trade_df[trade_df['Metric'].str.contains('won.total')]['Value'].values[0]
# lost_trades = trade_df[trade_df['Metric'].str.contains('lost.total')]['Value'].values[0]

# # Create a DataFrame for trade outcomes
# trade_outcomes_df = pd.DataFrame({
#     'Outcome': ['Won', 'Lost'],
#     'Count': [won_trades, lost_trades]
# })

# # Save trade outcomes to CSV
# trade_outcomes_df.to_csv(os.path.join(results_dir, "trade_outcomes.csv"), index=False)

# # Plot trade outcomes
# fig = px.pie(trade_outcomes_df, values='Count', names='Outcome', title='Trade Outcomes')
# fig.write_html(os.path.join(results_dir, "trade_outcomes.html"))
# fig.write_image(os.path.join(results_dir, "trade_outcomes.png"))
# fig.show()

# # 2. PnL (Profit and Loss)
# gross_pnl = trade_df[trade_df['Metric'].str.contains('pnl.gross.total')]['Value'].values[0]
# net_pnl = trade_df[trade_df['Metric'].str.contains('pnl.net.total')]['Value'].values[0]

# # Create a DataFrame for PnL
# pnl_df = pd.DataFrame({
#     'Type': ['Gross PnL', 'Net PnL'],
#     'Value': [gross_pnl, net_pnl]
# })

# # Save PnL data to CSV
# pnl_df.to_csv(os.path.join(results_dir, "pnl_data.csv"), index=False)

# # Plot PnL
# fig = px.bar(pnl_df, x='Type', y='Value', title='Gross vs Net PnL')
# fig.write_html(os.path.join(results_dir, "pnl_data.html"))
# fig.write_image(os.path.join(results_dir, "pnl_data.png"))
# fig.show()

# # 3. Streaks
# won_streak_current = trade_df[trade_df['Metric'].str.contains('streak.won.current')]['Value'].values[0]
# won_streak_longest = trade_df[trade_df['Metric'].str.contains('streak.won.longest')]['Value'].values[0]
# lost_streak_current = trade_df[trade_df['Metric'].str.contains('streak.lost.current')]['Value'].values[0]
# lost_streak_longest = trade_df[trade_df['Metric'].str.contains('streak.lost.longest')]['Value'].values[0]

# # Create a DataFrame for streaks
# streaks_df = pd.DataFrame({
#     'Streak Type': ['Won (Current)', 'Won (Longest)', 'Lost (Current)', 'Lost (Longest)'],
#     'Value': [won_streak_current, won_streak_longest, lost_streak_current, lost_streak_longest]
# })

# # Save streaks data to CSV
# streaks_df.to_csv(os.path.join(results_dir, "streaks_data.csv"), index=False)

# # Plot streaks
# fig = px.bar(streaks_df, x='Streak Type', y='Value', title='Winning and Losing Streaks')
# fig.write_html(os.path.join(results_dir, "streaks_data.html"))
# fig.write_image(os.path.join(results_dir, "streaks_data.png"))
# fig.show()

# # 4. Trade Length
# avg_trade_length = get_metric_value(trade_df, 'len.total.average')
# max_trade_length = get_metric_value(trade_df, 'len.total.max')

# # Create a DataFrame for trade length (only if metrics are available)
# if avg_trade_length is not None and max_trade_length is not None:
#     trade_length_df = pd.DataFrame({
#         'Metric': ['Average Trade Length', 'Max Trade Length'],
#         'Value': [avg_trade_length, max_trade_length]
#     })

#     # Save trade length data to CSV
#     trade_length_df.to_csv(os.path.join(results_dir, "trade_length_data.csv"), index=False)

#     # Plot trade length
#     fig = px.bar(trade_length_df, x='Metric', y='Value', title='Trade Length')
#     fig.write_html(os.path.join(results_dir, "trade_length_data.html"))
#     fig.write_image(os.path.join(results_dir, "trade_length_data.png"))
#     fig.show()
# else:
#     print("Trade length metrics not found in the analysis.")

# # Create a summary text file
# with open(os.path.join(results_dir, "summary.txt"), "w") as f:
#     f.write(f"Backtest Results Summary\n")
#     f.write(f"======================\n\n")
#     f.write(f"Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
#     f.write(f"Total Trades: {total_trades}\n")
#     f.write(f"Won Trades: {won_trades} ({won_trades/total_trades*100:.2f}%)\n")
#     f.write(f"Lost Trades: {lost_trades} ({lost_trades/total_trades*100:.2f}%)\n\n")
#     f.write(f"Gross PnL: {gross_pnl}\n")
#     f.write(f"Net PnL: {net_pnl}\n\n")
#     f.write(f"Current Win Streak: {won_streak_current}\n")
#     f.write(f"Longest Win Streak: {won_streak_longest}\n")
#     f.write(f"Current Loss Streak: {lost_streak_current}\n")
#     f.write(f"Longest Loss Streak: {lost_streak_longest}\n")

# print(f"All results saved to {results_dir}")

In [ ]:
import pandas as pd
import plotly.express as px
import os
import datetime
import sys
import subprocess

# Create directory for saving results
def create_results_dir(base_path=None):
    """
    Creates a directory for saving results based on current timestamp
    Returns the path to the created directory
    """
    # If no base path provided, use the directory where the notebook is located
    if base_path is None:
        base_path = os.path.dirname(os.path.abspath('Backtest.ipynb'))

    # create a output directory if not exists
    base_path = os.path.join(base_path, 'output')
    if not os.path.exists(base_path):
        os.makedirs(base_path)

    # create a selected stock directory if not exists
    base_path = os.path.join(base_path, selected)
    if not os.path.exists(base_path):
        os.makedirs(base_path)
    
    # Create a timestamp-based folder name
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = os.path.join(base_path, f"{selected}_backtest_results_{timestamp}")
    
    # Create the directory if it doesn't exist
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        
    return results_dir

# Function to flatten AutoOrderedDict
def flatten_dict(d, parent_key='', sep='.'):
    """
    Recursively flattens a nested dictionary.
    """
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, dict):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Function to safely extract a metric value
def get_metric_value(trade_df, metric_key):
    """
    Safely extracts a metric value from the DataFrame.
    Returns None if the metric is not found.
    """
    matches = trade_df[trade_df['Metric'].str.contains(metric_key)]
    if not matches.empty:
        return matches['Value'].values[0]
    return None

# Try to install kaleido if it's not already installed
try:
    import kaleido
    has_kaleido = True
except ImportError:
    has_kaleido = False
    print("Kaleido package not found. Attempting to install...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "kaleido"])
        has_kaleido = True
        print("Kaleido installed successfully!")
    except Exception as e:
        print(f"Could not install kaleido: {e}")
        print("Will continue without image export capability")

# Create results directory
results_dir = create_results_dir()
print(f"Results will be saved to: {results_dir}")

# Flatten the trade_analysis object
flat_trade_analysis = flatten_dict(trade_analysis)

# Convert the flattened dictionary to a DataFrame
trade_df = pd.DataFrame(list(flat_trade_analysis.items()), columns=['Metric', 'Value'])

# Save the full analysis to CSV
trade_df.to_csv(os.path.join(results_dir, f"{selected}_trade_analysis.csv"), index=False)
print(f"Trade analysis saved to CSV")

# Print the DataFrame
print(trade_df)

# 1. Total Trades
total_trades = trade_df[trade_df['Metric'].str.contains('total.total')]['Value'].values[0]
won_trades = trade_df[trade_df['Metric'].str.contains('won.total')]['Value'].values[0]
lost_trades = trade_df[trade_df['Metric'].str.contains('lost.total')]['Value'].values[0]

# Create a DataFrame for trade outcomes
trade_outcomes_df = pd.DataFrame({
    'Outcome': ['Won', 'Lost'],
    'Count': [won_trades, lost_trades]
})

# Save trade outcomes to CSV
trade_outcomes_df.to_csv(os.path.join(results_dir, f"{selected}_trade_outcomes.csv"), index=False)

# Plot trade outcomes
fig = px.pie(trade_outcomes_df, values='Count', names='Outcome', title='Trade Outcomes')
fig.write_html(os.path.join(results_dir, "trade_outcomes.html"))
if has_kaleido:
    try:
        fig.write_image(os.path.join(results_dir, "trade_outcomes.png"))
    except Exception as e:
        print(f"Warning: Could not save image: {e}")
fig.show()

# 2. PnL (Profit and Loss)
gross_pnl = trade_df[trade_df['Metric'].str.contains('pnl.gross.total')]['Value'].values[0]
net_pnl = trade_df[trade_df['Metric'].str.contains('pnl.net.total')]['Value'].values[0]

# Create a DataFrame for PnL
pnl_df = pd.DataFrame({
    'Type': ['Gross PnL', 'Net PnL'],
    'Value': [gross_pnl, net_pnl]
})

# Save PnL data to CSV
pnl_df.to_csv(os.path.join(results_dir, f"{selected}_pnl_data.csv"), index=False)

# Plot PnL
fig = px.bar(pnl_df, x='Type', y='Value', title='Gross vs Net PnL')
fig.write_html(os.path.join(results_dir, f"{selected}_pnl_data.html"))
if has_kaleido:
    try:
        fig.write_image(os.path.join(results_dir, f"{selected}_pnl_data.png"))
    except Exception as e:
        print(f"Warning: Could not save image: {e}")
fig.show()

# 3. Streaks
won_streak_current = trade_df[trade_df['Metric'].str.contains('streak.won.current')]['Value'].values[0]
won_streak_longest = trade_df[trade_df['Metric'].str.contains('streak.won.longest')]['Value'].values[0]
lost_streak_current = trade_df[trade_df['Metric'].str.contains('streak.lost.current')]['Value'].values[0]
lost_streak_longest = trade_df[trade_df['Metric'].str.contains('streak.lost.longest')]['Value'].values[0]

# Create a DataFrame for streaks
streaks_df = pd.DataFrame({
    'Streak Type': ['Won (Current)', 'Won (Longest)', 'Lost (Current)', 'Lost (Longest)'],
    'Value': [won_streak_current, won_streak_longest, lost_streak_current, lost_streak_longest]
})

# Save streaks data to CSV
streaks_df.to_csv(os.path.join(results_dir, f"{selected}_streaks_data.csv"), index=False)

# Plot streaks
fig = px.bar(streaks_df, x='Streak Type', y='Value', title='Winning and Losing Streaks')
fig.write_html(os.path.join(results_dir, f"{selected}_streaks_data.html"))
if has_kaleido:
    try:
        fig.write_image(os.path.join(results_dir, f"{selected}_streaks_data.png"))
    except Exception as e:
        print(f"Warning: Could not save image: {e}")
fig.show()

# 4. Trade Length
avg_trade_length = get_metric_value(trade_df, 'len.total.average')
max_trade_length = get_metric_value(trade_df, 'len.total.max')

# Create a DataFrame for trade length (only if metrics are available)
if avg_trade_length is not None and max_trade_length is not None:
    trade_length_df = pd.DataFrame({
        'Metric': ['Average Trade Length', 'Max Trade Length'],
        'Value': [avg_trade_length, max_trade_length]
    })

    # Save trade length data to CSV
    trade_length_df.to_csv(os.path.join(results_dir, f"{selected}_trade_length_data.csv"), index=False)

    # Plot trade length
    fig = px.bar(trade_length_df, x='Metric', y='Value', title='Trade Length')
    fig.write_html(os.path.join(results_dir, f"{selected}_trade_length_data.html"))
    if has_kaleido:
        try:
            fig.write_image(os.path.join(results_dir, f"{selected}_trade_length_data.png"))
        except Exception as e:
            print(f"Warning: Could not save image: {e}")
    fig.show()
else:
    print("Trade length metrics not found in the analysis.")

# Create a summary text file
with open(os.path.join(results_dir, f"{selected}_summary.txt"), "w") as f:
    f.write(f"Backtest Results Summary\n")
    f.write(f"======================\n\n")
    f.write(f"Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write(f"Total Trades: {total_trades}\n")
    f.write(f"Won Trades: {won_trades} ({won_trades/total_trades*100:.2f}%)\n")
    f.write(f"Lost Trades: {lost_trades} ({lost_trades/total_trades*100:.2f}%)\n\n")
    f.write(f"Gross PnL: {gross_pnl}\n")
    f.write(f"Net PnL: {net_pnl}\n\n")
    f.write(f"Current Win Streak: {won_streak_current}\n")
    f.write(f"Longest Win Streak: {won_streak_longest}\n")
    f.write(f"Current Loss Streak: {lost_streak_current}\n")
    f.write(f"Longest Loss Streak: {lost_streak_longest}\n")

print(f"All results saved to {results_dir}")

Results will be saved to: c:\Users\athabit\Desktop\misc\bhuv\PEAD_Regression\backtest\output\NVDA\NVDA_backtest_results_20250321_185142
Trade analysis saved to CSV
                    Metric  Value
0              total.total   39.0
1               total.open    0.0
2             total.closed   39.0
3       streak.won.current    1.0
4       streak.won.longest    4.0
..                     ...    ...
72       len.short.won.min    5.0
73    len.short.lost.total    4.0
74  len.short.lost.average    2.0
75      len.short.lost.max    3.0
76      len.short.lost.min    1.0

[77 rows x 2 columns]
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



Trade length metrics not found in the analysis.
All results saved to c:\Users\athabit\Desktop\misc\bhuv\PEAD_Regression\backtest\output\NVDA\NVDA_backtest_results_20250321_185142


In [ ]:
# import pandas as pd
# import plotly.express as px

# # Function to flatten AutoOrderedDict
# def flatten_dict(d, parent_key='', sep='.'):
#     """
#     Recursively flattens a nested dictionary.
#     """
#     items = []
#     for k, v in d.items():
#         new_key = f"{parent_key}{sep}{k}" if parent_key else k
#         if isinstance(v, dict):
#             items.extend(flatten_dict(v, new_key, sep=sep).items())
#         else:
#             items.append((new_key, v))
#     return dict(items)

# # Flatten the trade_analysis object
# flat_trade_analysis = flatten_dict(trade_analysis)

# # Convert the flattened dictionary to a DataFrame
# trade_df = pd.DataFrame(list(flat_trade_analysis.items()), columns=['Metric', 'Value'])

# # Print the DataFrame
# print(trade_df)

# # Visualize key metrics
# # 1. Total Trades
# total_trades = trade_df[trade_df['Metric'].str.contains('total.total')]['Value'].values[0]
# won_trades = trade_df[trade_df['Metric'].str.contains('won.total')]['Value'].values[0]
# lost_trades = trade_df[trade_df['Metric'].str.contains('lost.total')]['Value'].values[0]

# # Create a DataFrame for trade outcomes
# trade_outcomes_df = pd.DataFrame({
#     'Outcome': ['Won', 'Lost'],
#     'Count': [won_trades, lost_trades]
# })

# # Plot trade outcomes
# fig = px.pie(trade_outcomes_df, values='Count', names='Outcome', title='Trade Outcomes')
# fig.show()

# # 2. PnL (Profit and Loss)
# gross_pnl = trade_df[trade_df['Metric'].str.contains('pnl.gross.total')]['Value'].values[0]
# net_pnl = trade_df[trade_df['Metric'].str.contains('pnl.net.total')]['Value'].values[0]

# # Create a DataFrame for PnL
# pnl_df = pd.DataFrame({
#     'Type': ['Gross PnL', 'Net PnL'],
#     'Value': [gross_pnl, net_pnl]
# })

# # Plot PnL
# fig = px.bar(pnl_df, x='Type', y='Value', title='Gross vs Net PnL')
# fig.show()

# # 3. Streaks
# won_streak_current = trade_df[trade_df['Metric'].str.contains('streak.won.current')]['Value'].values[0]
# won_streak_longest = trade_df[trade_df['Metric'].str.contains('streak.won.longest')]['Value'].values[0]
# lost_streak_current = trade_df[trade_df['Metric'].str.contains('streak.lost.current')]['Value'].values[0]
# lost_streak_longest = trade_df[trade_df['Metric'].str.contains('streak.lost.longest')]['Value'].values[0]

# # Create a DataFrame for streaks
# streaks_df = pd.DataFrame({
#     'Streak Type': ['Won (Current)', 'Won (Longest)', 'Lost (Current)', 'Lost (Longest)'],
#     'Value': [won_streak_current, won_streak_longest, lost_streak_current, lost_streak_longest]
# })

# # Plot streaks
# fig = px.bar(streaks_df, x='Streak Type', y='Value', title='Winning and Losing Streaks')
# fig.show()

# # 4. Trade Length
# # Function to safely extract a metric value
# def get_metric_value(trade_df, metric_key):
#     """
#     Safely extracts a metric value from the DataFrame.
#     Returns None if the metric is not found.
#     """
#     matches = trade_df[trade_df['Metric'].str.contains(metric_key)]
#     if not matches.empty:
#         return matches['Value'].values[0]
#     return None

# # 4. Trade Length
# avg_trade_length = get_metric_value(trade_df, 'len.total.average')
# max_trade_length = get_metric_value(trade_df, 'len.total.max')

# # Create a DataFrame for trade length (only if metrics are available)
# if avg_trade_length is not None and max_trade_length is not None:
#     trade_length_df = pd.DataFrame({
#         'Metric': ['Average Trade Length', 'Max Trade Length'],
#         'Value': [avg_trade_length, max_trade_length]
#     })

#     # Plot trade length
#     fig = px.bar(trade_length_df, x='Metric', y='Value', title='Trade Length')
#     fig.show()
# else:
#     print("Trade length metrics not found in the analysis.")

In [ ]:
import numpy as np
import plotly.express as px
import pandas as pd
import os
import datetime
import sys
import subprocess

# Try to install kaleido if it's not already installed
try:
    import kaleido
    has_kaleido = True
except ImportError:
    has_kaleido = False
    print("Kaleido package not found. Attempting to install...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "kaleido"])
        has_kaleido = True
        print("Kaleido installed successfully!")
    except Exception as e:
        print(f"Could not install kaleido: {e}")
        print("Will continue without image export capability")

# Function to create a results directory
def create_results_dir(base_path=None):
    """
    Creates a directory for saving results based on current timestamp
    Returns the path to the created directory
    """
    # If no base path provided, use the directory where the notebook is located
    if base_path is None:
        base_path = os.path.dirname(os.path.abspath('Backtest.ipynb'))
    
    # Create a timestamp-based folder name
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = os.path.join(base_path, f"{selected}_portfolio_analysis_{timestamp}")
    
    # Create the directory if it doesn't exist
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        
    return results_dir

# Create results directory
results_dir = create_results_dir()
print(f"Portfolio analysis results will be saved to: {results_dir}")

# Assuming self.portfolio_value is a list of portfolio values over time
portfolio_values = thestrat.portfolio_value

# Calculate total return
initial_value = portfolio_values[0]
final_value = portfolio_values[-1]
total_return = (final_value - initial_value) / initial_value

# Calculate the number of years
num_days = len(portfolio_values)  # Assuming daily data
num_years = num_days / 252  # Assuming 252 trading days in a year

# Calculate annualized return
annual_return = (1 + total_return) ** (1 / num_years) - 1

# Print the annual return
print(f"Annual Return: {annual_return * 100:.2f}%")

# Create a DataFrame for portfolio values
portfolio_df = pd.DataFrame({
    'Date': price_df.index,  # Replace with actual dates
    'Portfolio Value': portfolio_values
})

# Save portfolio data to CSV
portfolio_df.to_csv(os.path.join(results_dir, f"{selected}_portfolio_values.csv"), index=False)
print(f"Portfolio data saved to CSV")

# Plot portfolio growth
fig = px.line(portfolio_df, x='Date', y='Portfolio Value', title='Portfolio Growth Over Time')
fig.write_html(os.path.join(results_dir, f"{selected}_portfolio_growth.html"))
if has_kaleido:
    try:
        fig.write_image(os.path.join(results_dir, f"{selected}_portfolio_growth.png"))
    except Exception as e:
        print(f"Warning: Could not save image: {e}")
fig.show()

# Calculate additional metrics
# Compute daily returns
portfolio_df['Daily_Return'] = portfolio_df['Portfolio Value'].pct_change()

# Calculate drawdown
portfolio_df['Peak'] = portfolio_df['Portfolio Value'].cummax()
portfolio_df['Drawdown'] = (portfolio_df['Portfolio Value'] - portfolio_df['Peak']) / portfolio_df['Peak']

# Calculate volatility (annualized)
volatility = portfolio_df['Daily_Return'].std() * np.sqrt(252)

# Calculate Sharpe ratio (assuming risk-free rate of 0% for simplicity)
sharpe_ratio = annual_return / volatility if volatility != 0 else 0

# Calculate maximum drawdown
max_drawdown = portfolio_df['Drawdown'].min()

# Save additional portfolio metrics to CSV
metrics_df = pd.DataFrame({
    'Metric': ['Initial Value', 'Final Value', 'Total Return', 'Annual Return', 
               'Volatility', 'Sharpe Ratio', 'Maximum Drawdown', 'Number of Trading Days'],
    'Value': [initial_value, final_value, total_return, annual_return, 
              volatility, sharpe_ratio, max_drawdown, num_days]
})

metrics_df.to_csv(os.path.join(results_dir, f"{selected}_portfolio_metrics.csv"), index=False)
print(f"Portfolio metrics saved to CSV")

# Plot drawdown
fig_drawdown = px.line(portfolio_df, x='Date', y='Drawdown', title='Portfolio Drawdown')
fig_drawdown.update_layout(yaxis_tickformat='.1%')
fig_drawdown.write_html(os.path.join(results_dir, f"{selected}_portfolio_drawdown.html"))
if has_kaleido:
    try:
        fig_drawdown.write_image(os.path.join(results_dir, f"{selected}_portfolio_drawdown.png"))
    except Exception as e:
        print(f"Warning: Could not save drawdown image: {e}")
fig_drawdown.show()

# Create a summary text file
with open(os.path.join(results_dir, f"{selected}_portfolio_summary.txt"), "w") as f:
    f.write(f"Portfolio Analysis Summary\n")
    f.write(f"=========================\n\n")
    f.write(f"Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write(f"Initial Portfolio Value: ${initial_value:.2f}\n")
    f.write(f"Final Portfolio Value: ${final_value:.2f}\n")
    f.write(f"Total Return: {total_return * 100:.2f}%\n")
    f.write(f"Annual Return: {annual_return * 100:.2f}%\n")
    f.write(f"Volatility (Annualized): {volatility * 100:.2f}%\n")
    f.write(f"Sharpe Ratio: {sharpe_ratio:.2f}\n")
    f.write(f"Maximum Drawdown: {max_drawdown * 100:.2f}%\n")
    f.write(f"Trading Period: {num_days} days ({num_years:.2f} years)\n")

print(f"All portfolio analysis results saved to {results_dir}")

Portfolio analysis results will be saved to: c:\Users\athabit\Desktop\misc\bhuv\PEAD_Regression\backtest\NVDA_portfolio_analysis_20250321_185142
Annual Return: 0.70%
Portfolio data saved to CSV
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



Portfolio metrics saved to CSV
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



All portfolio analysis results saved to c:\Users\athabit\Desktop\misc\bhuv\PEAD_Regression\backtest\NVDA_portfolio_analysis_20250321_185142


In [ ]:
# import numpy as np

# # Assuming self.portfolio_value is a list of portfolio values over time
# portfolio_values = thestrat.portfolio_value

# # Calculate total return
# initial_value = portfolio_values[0]
# final_value = portfolio_values[-1]
# total_return = (final_value - initial_value) / initial_value

# # Calculate the number of years
# num_days = len(portfolio_values)  # Assuming daily data
# num_years = num_days / 252  # Assuming 252 trading days in a year

# # Calculate annualized return
# annual_return = (1 + total_return) ** (1 / num_years) - 1

# # Print the annual return
# print(f"Annual Return: {annual_return * 100:.2f}%")

# import plotly.express as px
# import pandas as pd

# # Create a DataFrame for portfolio values
# portfolio_df = pd.DataFrame({
#     'Date': price_df.index,  # Replace with actual dates
#     'Portfolio Value': portfolio_values
# })

# # Plot portfolio growth
# fig = px.line(portfolio_df, x='Date', y='Portfolio Value', title='Portfolio Growth Over Time')
# fig.show()

In [ ]:
# drawdown_series = drawdown['drawdown']
# print(drawdown_series)

# # Create a DataFrame for drawdown
# drawdown_df = pd.DataFrame({
#     'Time': range(len(drawdown_series)),
#     'Drawdown (%)': drawdown_series
# })

# # Plot drawdown curve
# fig = px.line(drawdown_df, x='Time', y='Drawdown (%)', title='Drawdown Over Time')
# fig.show()

In [ ]:
# import pandas as pd
# import plotly.express as px

# # Convert trade_details to DataFrame
# trade_df = pd.DataFrame(thestrat.trade_details)

# # Classify outcomes based on exit reason
# def categorize_outcome(exit_reason):
#     if exit_reason == 'Take Profit':
#         return 'Won'
#     elif exit_reason == 'Stop Loss':
#         return 'Lost'
#     elif exit_reason == 'Timeout':
#         return 'Exited'
#     else:
#         return 'Unknown'  # Fallback for unexpected values

# trade_df['Outcome'] = trade_df['Exit Reason'].apply(categorize_outcome)

# # Group by direction (Long/Short) and outcome (Won/Lost/Exited)
# grouped = trade_df.groupby(['Direction', 'Outcome']).size().reset_index(name='Count')

# # Pivot the data for better visualization
# pivot_df = grouped.pivot(index='Direction', columns='Outcome', values='Count').fillna(0)

# # Print the pivot table
# print(pivot_df)

# # Plot the results
# fig = px.bar(
#     grouped,
#     x='Direction',
#     y='Count',
#     color='Outcome',
#     barmode='group',
#     title='Trade Outcomes by Direction (Long/Short) and Result (Won/Lost/Exited)',
#     labels={'Count': 'Number of Trades', 'Direction': 'Trade Direction'}
# )
# fig.show()

In [ ]:
import pandas as pd
import plotly.express as px
import os
import datetime
import sys
import subprocess

# Try to install kaleido if it's not already installed
try:
    import kaleido
    has_kaleido = True
except ImportError:
    has_kaleido = False
    print("Kaleido package not found. Attempting to install...")
    try:
        subprocess.check_call([sys.executable, "-m", "pip", "install", "kaleido"])
        has_kaleido = True
        print("Kaleido installed successfully!")
    except Exception as e:
        print(f"Could not install kaleido: {e}")
        print("Will continue without image export capability")

# Function to create a results directory
def create_results_dir(base_path=None):
    """
    Creates a directory for saving results based on current timestamp
    Returns the path to the created directory
    """
    # If no base path provided, use the directory where the notebook is located
    if base_path is None:
        base_path = os.path.dirname(os.path.abspath('Backtest.ipynb'))
    
    # Create a timestamp-based folder name
    timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
    results_dir = os.path.join(base_path, f"{selected}_equity_curve_{timestamp}")
    
    # Create the directory if it doesn't exist
    if not os.path.exists(results_dir):
        os.makedirs(results_dir)
        
    return results_dir

# Create results directory
results_dir = create_results_dir()
print(f"Equity curve results will be saved to: {results_dir}")

portfolio_value = thestrat.portfolio_value

# Calculate cumulative returns over time
initial_cash = cerebro.broker.startingcash
cumulative_returns = [(value / initial_cash - 1) * 100 for value in portfolio_value]  # In percentage
time_index = price_df.index

# Create a DataFrame for cumulative returns
returns_df = pd.DataFrame({
    'Time': time_index,  # Use the index of price_df
    'Cumulative Return (%)': cumulative_returns
})

# Save equity curve data to CSV
returns_df.to_csv(os.path.join(results_dir, f"{selected}_equity_curve_data.csv"), index=False)
print(f"Equity curve data saved to CSV")

# Plot equity curve
fig = px.line(returns_df, x='Time', y='Cumulative Return (%)', title='Equity Curve')
fig.write_html(os.path.join(results_dir, f"{selected}_equity_curve.html"))
if has_kaleido:
    try:
        fig.write_image(os.path.join(results_dir, f"{selected}_equity_curve.png"))
    except Exception as e:
        print(f"Warning: Could not save image: {e}")
fig.show()

# Calculate additional metrics
# Monthly returns
if len(returns_df) > 30:  # Ensure we have enough data
    returns_df['Month'] = returns_df['Time'].dt.to_period('M')
    monthly_returns = returns_df.groupby('Month').last()['Cumulative Return (%)'].pct_change() * 100
    monthly_returns_df = pd.DataFrame(monthly_returns).reset_index()
    monthly_returns_df.columns = ['Month', 'Monthly Return (%)']
    
    # Convert Period to string for better compatibility
    monthly_returns_df['Month'] = monthly_returns_df['Month'].astype(str)
    
    # Save monthly returns to CSV
    monthly_returns_df.to_csv(os.path.join(results_dir, f"{selected}_monthly_returns.csv"), index=False)
    
    # Plot monthly returns
    if len(monthly_returns_df) > 1:  # Ensure we have at least 2 months of data
        fig_monthly = px.bar(monthly_returns_df, x='Month', y='Monthly Return (%)', 
                             title='Monthly Returns')
        fig_monthly.write_html(os.path.join(results_dir, f"{selected}_monthly_returns.html"))
        if has_kaleido:
            try:
                fig_monthly.write_image(os.path.join(results_dir, f"{selected}_monthly_returns.png"))
            except Exception as e:
                print(f"Warning: Could not save monthly returns image: {e}")
        fig_monthly.show()

# Calculate drawdown
if len(returns_df) > 0:
    returns_df['Peak'] = returns_df['Cumulative Return (%)'].cummax()
    returns_df['Drawdown'] = returns_df['Cumulative Return (%)'] - returns_df['Peak']
    
    # Save drawdown data
    drawdown_df = returns_df[['Time', 'Drawdown']]
    drawdown_df.to_csv(os.path.join(results_dir, f"{selected}_drawdown_data.csv"), index=False)
    
    # Plot drawdown
    fig_drawdown = px.line(returns_df, x='Time', y='Drawdown', title='Drawdown (%)')
    fig_drawdown.write_html(os.path.join(results_dir, "drawdown.html"))
    if has_kaleido:
        try:
            fig_drawdown.write_image(os.path.join(results_dir, f"{selected}_drawdown.png"))
        except Exception as e:
            print(f"Warning: Could not save drawdown image: {e}")
    fig_drawdown.show()

# Calculate summary metrics
max_return = max(cumulative_returns) if cumulative_returns else 0
min_return = min(cumulative_returns) if cumulative_returns else 0
max_drawdown = returns_df['Drawdown'].min() if 'Drawdown' in returns_df.columns else 0
final_return = cumulative_returns[-1] if cumulative_returns else 0

# Create a summary text file
with open(os.path.join(results_dir, f"{selected}_equity_curve_summary.txt"), "w") as f:
    f.write(f"Equity Curve Analysis Summary\n")
    f.write(f"===========================\n\n")
    f.write(f"Generated on: {datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")
    f.write(f"Initial Investment: ${initial_cash:.2f}\n")
    f.write(f"Final Value: ${portfolio_value[-1]:.2f}\n")
    f.write(f"Final Return: {final_return:.2f}%\n")
    f.write(f"Maximum Return: {max_return:.2f}%\n")
    f.write(f"Maximum Drawdown: {max_drawdown:.2f}%\n")
    f.write(f"Trading Period: {len(cumulative_returns)} days\n")

print(f"All equity curve results saved to {results_dir}")

Equity curve results will be saved to: c:\Users\athabit\Desktop\misc\bhuv\PEAD_Regression\backtest\NVDA_equity_curve_20250321_185143
Equity curve data saved to CSV
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido



All equity curve results saved to c:\Users\athabit\Desktop\misc\bhuv\PEAD_Regression\backtest\NVDA_equity_curve_20250321_185143


In [ ]:
portfolio_value = thestrat.portfolio_value

# Calculate cumulative returns over time
initial_cash = cerebro.broker.startingcash
cumulative_returns = [(value / initial_cash - 1) * 100 for value in portfolio_value]  # In percentage
time_index = price_df.index
# Create a DataFrame for cumulative returns
returns_df = pd.DataFrame({
    'Time': time_index,  # Use the index of price_df
    'Cumulative Return (%)': cumulative_returns
})

# Plot equity curve
fig = px.line(returns_df, x='Time', y='Cumulative Return (%)', title='Equity Curve')
fig.show()


In [ ]:
peak_value = 0
drawdown = []
for value in portfolio_value:
    if value > peak_value:
        peak_value = value
    current_drawdown = (peak_value - value) / peak_value * 100  # Drawdown in percentage
    drawdown.append(current_drawdown)

# Create a DataFrame for drawdown
drawdown_df = pd.DataFrame({
    'Time': time_index,  # Use the index of price_df
    'Drawdown (%)': drawdown
})

# Plot drawdown curve
fig = px.line(drawdown_df, x='Time', y='Drawdown (%)', title='Drawdown Over Time')
fig.show()

In [ ]:
import plotly.graph_objects as go
trade_details = thestrat.trade_details

# Convert to a DataFrame
trades_df = pd.DataFrame(trade_details)

import plotly.graph_objects as go

# Define a modern color palette
header_color = 'rgb(56, 90, 93)'  # Dark teal
cell_color = 'rgb(227, 235, 236)'  # Light gray
font_color = 'rgb(0, 0, 0)'  # Black
border_color = 'rgb(150, 150, 150)'  # Gray

# Create an interactive table
fig = go.Figure(data=[go.Table(
    header=dict(
        values=list(trades_df.columns),  # Column headers
        fill_color=header_color,         # Header background color
        font=dict(color=font_color, size=14, family='Arial'),  # Header font
        align='left',                    # Header alignment
        line=dict(color=border_color, width=1)  # Header border
    ),
    cells=dict(
        values=[trades_df[col] for col in trades_df.columns],  # Cell values
        fill_color=cell_color,           # Cell background color
        font=dict(color=font_color, size=12, family='Arial'),  # Cell font
        align='left',                    # Cell alignment
        line=dict(color=border_color, width=1)  # Cell border
    ))
])

# Update layout for better appearance
fig.update_layout(
    title={
        'text': '<b>Trade Details</b>',  # Bold title
        'y': 0.95,  # Title position
        'x': 0.5,   # Center the title
        'yanchor': 'top',
        'font': dict(size=20, family='Arial', color=header_color)  # Title font
    },
    margin=dict(l=20, r=20, t=60, b=20),  # Add margins
    paper_bgcolor='rgb(248, 248, 248)',   # Light gray background
    plot_bgcolor='rgb(248, 248, 248)',    # Light gray background
)

# Show the table
fig.show()


AttributeError: 'Lines_LineSeries_LineIterator_DataAccessor_Strateg' object has no attribute 'trade_details'